In [2]:
!pip install  pyspark

     |████████████████████████████████| 212.4 MB 62 kB/s 
     |████████████████████████████████| 198 kB 64.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=38ff3d9ecd19b63861bcd04d34b040db3cc0e0677cd1b8440fb951d589dc4015
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [81]:
# примонтировать Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from pyspark.sql  import  SparkSession

In [9]:
#создаем сессию в pyspark
spark = SparkSession.builder.getOrCreate()

In [85]:
df  = spark.read.format('csv').option('header', 'true').load('/content/drive/MyDrive/cars.csv')

In [91]:
df.show(5)

+-----------------+----------+------------+------+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------------+
|manufacturer_name|model_name|transmission| color|odometer_value|year_produced|engine_fuel|engine_has_gas|engine_type|engine_capacity|body_type|has_warranty|state|drivetrain|price_usd|is_exchangeable|location_region|number_of_photos|up_counter|feature_0|feature_1|feature_2|feature_3|feature_4|feature_5|feature_6|feature_7|feature_8|feature_9|duration_listed|
+-----------------+----------+------------+------+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+----

Смотрим первую строку в транспонированном виде

In [94]:
df.show(1 , vertical=True)

-RECORD 0-------------------------
 manufacturer_name | Subaru       
 model_name        | Outback      
 transmission      | automatic    
 color             | silver       
 odometer_value    | 190000       
 year_produced     | 2010         
 engine_fuel       | gasoline     
 engine_has_gas    | False        
 engine_type       | gasoline     
 engine_capacity   | 2.5          
 body_type         | universal    
 has_warranty      | False        
 state             | owned        
 drivetrain        | all          
 price_usd         | 10900.0      
 is_exchangeable   | False        
 location_region   | Минская обл. 
 number_of_photos  | 9            
 up_counter        | 13           
 feature_0         | False        
 feature_1         | True         
 feature_2         | True         
 feature_3         | True         
 feature_4         | False        
 feature_5         | True         
 feature_6         | False        
 feature_7         | True         
 feature_8         |

Выбор  полей SELECT

In [98]:
df.select('manufacturer_name', 'model_name').show(5)
df.select(df['manufacturer_name'], df['model_name']).show(5)

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|           Subaru|   Outback|
|           Subaru|   Outback|
|           Subaru|  Forester|
|           Subaru|   Impreza|
|           Subaru|    Legacy|
+-----------------+----------+
only showing top 5 rows

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|           Subaru|   Outback|
|           Subaru|   Outback|
|           Subaru|  Forester|
|           Subaru|   Impreza|
|           Subaru|    Legacy|
+-----------------+----------+
only showing top 5 rows



In [101]:
import pyspark.sql.functions as f 
df.select(f.col('manufacturer_name'), f.col('model_name')).show(5)

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|           Subaru|   Outback|
|           Subaru|   Outback|
|           Subaru|  Forester|
|           Subaru|   Impreza|
|           Subaru|    Legacy|
+-----------------+----------+
only showing top 5 rows



**.filter()**

Метод .filter() принимает условия для фильтрации:

In [105]:
df.select('manufacturer_name', 'model_name')\
  .filter('manufacturer_name = "Audi"').show(5)

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|             Audi|        Q7|
|             Audi|        TT|
|             Audi|       100|
|             Audi|        A6|
|             Audi|        Q3|
+-----------------+----------+
only showing top 5 rows



In [112]:
#доп  фильтры  and 
df.select('manufacturer_name', 'model_name', 'transmission')\
  .filter('manufacturer_name =="Audi" and transmission == "mechanical"')\
  .show(5)
# или так 
df.select('manufacturer_name', 'model_name', 'transmission')\
  .filter('manufacturer_name == "Audi"')\
  .filter('transmission == "mechanical"')\
  .show(5)

+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|             Audi|       100|  mechanical|
|             Audi|A6 Allroad|  mechanical|
|             Audi|       100|  mechanical|
|             Audi|        A4|  mechanical|
|             Audi|        80|  mechanical|
+-----------------+----------+------------+
only showing top 5 rows

+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|             Audi|       100|  mechanical|
|             Audi|A6 Allroad|  mechanical|
|             Audi|       100|  mechanical|
|             Audi|        A4|  mechanical|
|             Audi|        80|  mechanical|
+-----------------+----------+------------+
only showing top 5 rows



In [114]:
#в формате  фильтрации  ниже можно запихивать питоновские переменные 
NAME = 'Audi'
df.select('manufacturer_name', 'model_name', 'transmission')\
  .filter(f.col('manufacturer_name') == NAME)\
  .filter(f.col('transmission') == 'mechanical')\
  .show(5)

+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|             Audi|       100|  mechanical|
|             Audi|A6 Allroad|  mechanical|
|             Audi|       100|  mechanical|
|             Audi|        A4|  mechanical|
|             Audi|        80|  mechanical|
+-----------------+----------+------------+
only showing top 5 rows



**.count()**

Подсчет кол-ва строк:


In [115]:
df.count()

38531

уникальные строки  по колонке 

In [116]:
df.select("manufacturer_name").distinct().count()

55

**.groupBy() and .orderBy()**

Группировка и агрегация (аналог GROUP BY в SQL):

In [126]:
# сгрупировать по manufacturer_name и посчитать кол-во каждого
df.groupby('manufacturer_name')\
  .count()\
  .distinct()\
  .orderBy('count')\
  .show(5)

#или с заданной сортировкой # desc/asc
df.groupby('manufacturer_name')\
  .count()\
  .distinct()\
  .orderBy(f.col('count').desc())\
  .show(5)
# еще один способ сортировки указать  минус  
df.groupby('manufacturer_name')\
  .count()\
  .distinct()\
  .orderBy( - f.col('count'))\
  .show(5)

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|          Lincoln|   36|
|       Great Wall|   36|
|              ЗАЗ|   42|
|          Pontiac|   42|
|         Cadillac|   43|
+-----------------+-----+
only showing top 5 rows

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Volkswagen| 4243|
|             Opel| 2759|
|              BMW| 2610|
|             Ford| 2566|
|          Renault| 2493|
+-----------------+-----+
only showing top 5 rows

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Volkswagen| 4243|
|             Opel| 2759|
|              BMW| 2610|
|             Ford| 2566|
|          Renault| 2493|
+-----------------+-----+
only showing top 5 rows




**.withColumnRenamed() and .withColumn()** 

Переименовать существующую колонку:

In [127]:
df.withColumnRenamed('manufacturer_name', 'manufacturer')\
  .select('manufacturer')\
  .show(5)

+------------+
|manufacturer|
+------------+
|      Subaru|
|      Subaru|
|      Subaru|
|      Subaru|
|      Subaru|
+------------+
only showing top 5 rows



Создать новую колонку. Первый аргумент это название новой колонки, второй агрумент это выражение (обязательно использовать col() если ссылаемся на другую колонку):

In [131]:
df.withColumn('next_year', f.col('year_produced')+1)\
  .select('next_year','year_produced')\
  .show(5)

+---------+-------------+
|next_year|year_produced|
+---------+-------------+
|   2011.0|         2010|
|   2003.0|         2002|
|   2002.0|         2001|
|   2000.0|         1999|
|   2002.0|         2001|
+---------+-------------+
only showing top 5 rows



**.printSchema() and .describe()**


Вывести схему датафрейма (типы колонок):

In [132]:
df.printSchema()

root
 |-- manufacturer_name: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- color: string (nullable = true)
 |-- odometer_value: string (nullable = true)
 |-- year_produced: string (nullable = true)
 |-- engine_fuel: string (nullable = true)
 |-- engine_has_gas: string (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- engine_capacity: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- has_warranty: string (nullable = true)
 |-- state: string (nullable = true)
 |-- drivetrain: string (nullable = true)
 |-- price_usd: string (nullable = true)
 |-- is_exchangeable: string (nullable = true)
 |-- location_region: string (nullable = true)
 |-- number_of_photos: string (nullable = true)
 |-- up_counter: string (nullable = true)
 |-- feature_0: string (nullable = true)
 |-- feature_1: string (nullable = true)
 |-- feature_2: string (nullable = true)
 |-- feature_3: string (nullable = true)


In [134]:
df.describe().show()

+-------+-----------------+------------------+------------+------+------------------+------------------+-------------+--------------+-----------+-----------------+---------+------------+---------+----------+-----------------+---------------+----------------+-----------------+------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+
|summary|manufacturer_name|        model_name|transmission| color|    odometer_value|     year_produced|  engine_fuel|engine_has_gas|engine_type|  engine_capacity|body_type|has_warranty|    state|drivetrain|        price_usd|is_exchangeable| location_region| number_of_photos|        up_counter|feature_0|feature_1|feature_2|feature_3|feature_4|feature_5|feature_6|feature_7|feature_8|feature_9|   duration_listed|
+-------+-----------------+------------------+------------+------+------------------+------------------+-------------+--------------+-----------+-----------------+-------

In [ ]:
#оставнока сессии
spark.stop()